<a href="https://colab.research.google.com/github/KKSTB/COMM061/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

# If run in colab instead of local
if 'google.colab' in sys.modules:
    # Install any dependencies
    !pip install datasets transformers seqeval huggingface_hub
    !pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=d51e07ae2f6d8e369c05e2a198afeb85cd5976cc4dcdc9fd879c8eb21b0cf8fa
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resol

In [2]:
from datasets import load_dataset
datasets = load_dataset("surrey-nlp/PLOD-CW-25")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/268 [00:00<?, ?B/s]

PLOD-CW-25-Train.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

PLOD-CW-25-Test.parquet:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

PLOD-CW-25-Val.parquet:   0%|          | 0.00/35.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/150 [00:00<?, ? examples/s]

In [3]:
label_list = ['O', 'B-AC', 'B-LF', 'I-LF']

labels_vocab = {k:v for v,k in enumerate(label_list)}

labels_vocab_reverse = {v:k for k,v in labels_vocab.items()}

In [4]:
def convert_ner_tags_to_int(dataset):
    dataset['ner_tag_ids'] = [labels_vocab[tag] for tag in dataset['ner_tags']]
    return dataset

datasets = datasets.map(convert_ner_tags_to_int)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") ## We use the BERT tokenizer

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [6]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [7]:
label_all_tokens = False

In [8]:
def tokenize_and_align_labels(dataset):
    tokenized_inputs = tokenizer(dataset["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(dataset["ner_tag_ids"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens(i.e.[CLS],[SEP]) have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [9]:
datasets_tokenized = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
args = TrainingArguments(                   ## This is where all your hyperparameters are, reduce *_batch_size if you are working with a GPU which has limited memory.
    "PLOD-ner",
    # evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16, # you can change the batch size according to the computational resources
    num_train_epochs=1,   ## may not be enough for a large data, increase this to 5 later in your experiments.
    weight_decay=0.01,
    push_to_hub=False,
    eval_steps=10000,
    save_steps=10000,
    report_to="none",

    #learning_rate is a hyperparameter that controls how quickly the model adapts to the training data. It specifies the size of the step taken at each iteration of the training process in the direction of minimizing the loss. A smaller learning rate results in slower but more precise convergence, while a larger learning rate can lead to faster convergence, but may result in overshooting the optimal solution.
    #per_device_train_batch_size :This parameter specifies how many training examples should be processed together in one forward/backward pass.
    #weight_decay: A regularization term added to the loss function to prevent overfitting. It controls the amount of regularization applied to the weights of the model during training.
    #push_to_hub: A flag that specifies whether to push the trained model to the HuggingFace model hub after training. The model hub is a repository of pre-trained models that can be easily shared and used by the community.
    #eval_steps: The number of training steps between each evaluation of the model on the validation set. This parameter is used to speed up the training process by evaluating the model less frequently.
    #num_train_epochs: The number of times to iterate over the entire training set during training. One epoch means one pass over the entire dataset. Increasing this value may improve the performance of the model, but also increases the training time.



)

In [12]:
import evaluate
metric = evaluate.load("seqeval")

In [13]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [14]:
trainer = Trainer(
    model,
    args,
    train_dataset=datasets_tokenized['train'],
    eval_dataset=datasets_tokenized['validation'],
    data_collator=DataCollatorForTokenClassification(tokenizer),
    compute_metrics=compute_metrics
)

In [15]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=63, training_loss=0.43781304737878224, metrics={'train_runtime': 71.5743, 'train_samples_per_second': 27.943, 'train_steps_per_second': 0.88, 'total_flos': 215818692353280.0, 'train_loss': 0.43781304737878224, 'epoch': 1.0})

In [16]:
trainer.evaluate()

{'eval_loss': 0.2953207790851593,
 'eval_precision': 0.7037837837837838,
 'eval_recall': 0.7997542997542998,
 'eval_f1': 0.7487061529614721,
 'eval_accuracy': 0.9007328447701533,
 'eval_runtime': 1.2709,
 'eval_samples_per_second': 118.029,
 'eval_steps_per_second': 7.869,
 'epoch': 1.0}

In [17]:
predictions, labels, _ = trainer.predict(datasets_tokenized['test'])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'AC': {'precision': np.float64(0.8052391799544419),
  'recall': np.float64(0.8870765370138017),
  'f1': np.float64(0.844179104477612),
  'number': np.int64(797)},
 'LF': {'precision': np.float64(0.5669421487603306),
  'recall': np.float64(0.7116182572614108),
  'f1': np.float64(0.6310947562097516),
  'number': np.int64(482)},
 'overall_precision': np.float64(0.708024275118004),
 'overall_recall': np.float64(0.8209538702111024),
 'overall_f1': np.float64(0.7603186097031137),
 'overall_accuracy': 0.9164473042800039}

In [18]:
labels[0]

array([-100,    0,    0, -100, -100, -100,    0,    0, -100, -100, -100,
          0,    0, -100,    0, -100, -100, -100, -100, -100,    0, -100,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, -100,    0,    0,    0, -100, -100, -100,    0,
          1, -100, -100,    0,    0,    0,    0,    0,    1, -100, -100,
          0,    0,    0,    0,    0,    0,    0,    0,    0, -100,    0,
       -100, -100, -100, -100, -100,    0, -100,    0,    0,    0,    0,
          0,    0,    0,    2, -100, -100,    3, -100,    0,    2, -100,
       -100, -100, -100, -100, -100,    3, -100,    3,    0,    1,    0,
          0, -100,    0,    0, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -1

In [19]:
datasets_tokenized['test'][0]

{'tokens': ['Within',
  'amoebae',
  'and',
  'macrophages',
  ',',
  'L.',
  'pneumophila',
  'replicates',
  'with',
  'a',
  'similar',
  'mechanism',
  'by',
  'forming',
  'a',
  'membrane',
  '-',
  'bound',
  'compartment',
  ',',
  'the',
  'Legionella',
  '-',
  'containing',
  'vacuole',
  '(',
  'LCV',
  ')',
  '.',
  'Formation',
  'of',
  'the',
  'LCV',
  'is',
  'not',
  'well',
  'defined',
  ',',
  'but',
  'requires',
  'that',
  'L.',
  'pneumophila',
  'injects',
  'proteins',
  'into',
  'the',
  'host',
  'cell',
  'via',
  'the',
  'intracellular',
  'multiplication',
  '(',
  'Icm)/defective',
  'organelle',
  'trafficking',
  '(',
  'Dot',
  ')',
  'secretion',
  'system',
  '.'],
 'pos_tags': ['ADP',
  'NOUN',
  'CCONJ',
  'NOUN',
  'PUNCT',
  'PROPN',
  'PROPN',
  'VERB',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'VERB',
  'DET',
  'NOUN',
  'PUNCT',
  'VERB',
  'NOUN',
  'PUNCT',
  'DET',
  'PROPN',
  'PUNCT',
  'VERB',
  'NOUN',
  'PUNCT',
  'PROPN',
